In [2]:
import patoolib
import os
import pandas as pd

In [3]:
os.listdir()

['archive.zip', 'Assignment.ipynb', 'kafka_spark']

In [4]:
patoolib.extract_archive("archive.zip")

INFO patool: Extracting archive.zip ...
INFO patool: ... archive.zip extracted to `archive' (multiple files in root).


'archive'

In [5]:
os.listdir("archive")

['test_dataset.csv', 'train_dataset.csv']

In [6]:
pd.options.display.max_columns = None

In [13]:
df = pd.read_csv("archive/train_dataset.csv")
df

,url,label,source,url_has_login,url_has_client,url_has_server,url_has_admin,url_has_ip,url_isshorted,url_len,url_entropy,url_hamming_1,url_hamming_00,url_hamming_10,url_hamming_01,url_hamming_11,url_2bentropy,url_3bentropy,url_count_dot,url_count_https,url_count_http,url_count_perc,url_count_hyphen,url_count_www,url_count_atrate,url_count_hash,url_count_semicolon,url_count_underscore,url_count_ques,url_count_equal,url_count_amp,url_count_letter,url_count_digit,url_count_sensitive_financial_words,url_count_sensitive_words,url_nunique_chars_ratio,path_len,path_count_no_of_dir,path_count_no_of_embed,path_count_zero,path_count_pertwent,path_has_any_sensitive_words,path_count_lower,path_count_upper,path_count_nonascii,path_has_singlechardir,path_has_upperdir,query_len,query_count_components,pdomain_len,pdomain_count_hyphen,pdomain_count_atrate,pdomain_count_non_alphanum,pdomain_count_digit,tld_len,tld,tld_is_sus,pdomain_min_distance,subdomain_len,subdomain_count_dot
0,irs-profilepaymentservice.com/home,1,phishtank,0,0,0,0,0,0,34,4.010134,0.540441,0.106618,0.205882,0.286765,0.194853,1.790285,2.360976,1,0,0,0,1,0,0,0,0,0,0,0,0,31,0,1,0,0.558824,4,1,0,0,0,0,31,0,0,0,0,0,0,0,0,0,0,0,3,com,0,17,0,0
1,cpuggsukabumi.id,0,majestic_million,0,0,0,0,0,0,16,3.452820,0.531250,0.109375,0.203125,0.296875,0.171875,1.773100,2.359953,1,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0.750000,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0,2,id,1,10,0,0
2,members.tripod.com/~don_rc/ring.htm,0,data_clean_test_mendel,0,0,0,0,0,0,35,4.000432,0.553571,0.096429,0.207143,0.271429,0.200000,1.770911,2.361831,3,0,0,0,0,0,0,0,0,1,0,0,0,28,0,0,0,0.514286,7,2,0,0,0,0,28,0,0,0,0,0,0,0,0,0,0,0,3,com,0,2,7,0
3,optuswebmailadminprovider.weebly.com/,1,phishtank,0,0,0,1,0,0,37,4.175273,0.537162,0.108108,0.206081,0.280405,0.189189,1.785399,2.364579,2,0,0,0,0,0,0,0,0,0,0,0,0,34,0,0,0,0.540541,0,1,0,0,0,0,34,0,0,0,0,0,0,0,0,0,0,0,3,com,0,3,25,0
4,topoz.com.pl,0,dmoz_harvard,0,0,0,0,0,0,12,2.855389,0.562500,0.093750,0.187500,0.239583,0.229167,1.753957,2.243520,2,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0.666667,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,6,com.pl,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6728843,detektei-ivanov.weebly.com,0,dmoz_harvard,0,0,0,0,0,0,26,3.869300,0.543269,0.096154,0.221154,0.298077,0.187500,1.779616,2.355990,2,0,0,0,1,0,0,0,0,0,0,0,0,23,0,0,0,0.653846,0,0,0,0,0,0,23,0,0,0,0,0,0,0,0,0,0,0,3,com,0,3,15,0
6728844,chase.trustbnkdirect.com/myaccount/access.php?...,1,ALL-phishing-links,0,0,0,0,0,1,113,4.762367,0.478982,0.164823,0.183628,0.255531,0.169248,1.814445,2.390548,3,0,0,0,0,0,0,0,0,0,1,3,2,71,31,0,2,0.292035,9,2,0,0,0,1,40,0,0,0,0,67,3,0,0,0,0,0,3,com,0,10,5,0
6728845,image.gsfc.nasa.gov,0,domcop,0,0,0,0,0,0,19,3.391893,0.546053,0.118421,0.184211,0.269737,0.203947,1.791784,2.325775,3,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,0,0.631579,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,0,3,gov,0,2,10,1
6728846,unregistered-device.cc/,1,phishtank,0,0,0,0,0,0,23,3.551191,0.521739,0.119565,0.217391,0.298913,0.168478,1.798629,2.412825,1,0,0,0,1,0,0,0,0,0,0,0,0,20,0,0,1,0.608696,0,1,0,0,0,1,20,0,0,0,0,0,0,0,0,0,0,0,2,cc,1,13,0,0


In [14]:
df.describe()

,label,url_has_login,url_has_client,url_has_server,url_has_admin,url_has_ip,url_isshorted,url_len,url_entropy,url_hamming_1,url_hamming_00,url_hamming_10,url_hamming_01,url_hamming_11,url_2bentropy,url_3bentropy,url_count_dot,url_count_https,url_count_http,url_count_perc,url_count_hyphen,url_count_www,url_count_atrate,url_count_hash,url_count_semicolon,url_count_underscore,url_count_ques,url_count_equal,url_count_amp,url_count_letter,url_count_digit,url_count_sensitive_financial_words,url_count_sensitive_words,url_nunique_chars_ratio,path_len,path_count_no_of_dir,path_count_no_of_embed,path_count_zero,path_count_pertwent,path_has_any_sensitive_words,path_count_lower,path_count_upper,path_count_nonascii,path_has_singlechardir,path_has_upperdir,query_len,query_count_components,pdomain_len,pdomain_count_hyphen,pdomain_count_atrate,pdomain_count_non_alphanum,pdomain_count_digit,tld_len,tld_is_sus,pdomain_min_distance,subdomain_len,subdomain_count_dot
count,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06,6.728848e+06
mean,2.148470e-01,1.562987e-02,2.001531e-03,2.242137e-03,3.648916e-03,3.619342e-03,5.041279e-02,2.375615e+01,3.526806e+00,5.344584e-01,1.103242e-01,2.033504e-01,2.761491e-01,1.928551e-01,1.782706e+00,2.345017e+00,1.512207e+00,1.699697e-03,2.609808e-03,1.876517e-02,2.804226e-01,5.354408e-03,2.390454e-03,1.038365e-03,3.756364e-03,4.301687e-02,2.688008e-02,5.232471e-02,2.386174e-02,1.959744e+01,1.563713e+00,1.297919e-02,8.175114e-02,6.888611e-01,1.615914e+00,5.961362e-01,2.900942e-04,1.515979e-01,1.103369e-02,6.048807e-02,1.832744e+01,1.591642e-01,1.422383e-03,7.323690e-03,7.015614e-03,1.763952e+00,4.846595e-02,7.831950e-05,8.916831e-07,2.972277e-07,1.263218e-05,2.615604e-05,3.102813e+00,8.032742e-02,6.647128e+00,2.272318e+00,4.740574e-02
std,4.107162e-01,1.240386e-01,4.469369e-02,4.729810e-02,6.029595e-02,6.005200e-02,2.187952e-01,3.337858e+01,4.597844e-01,2.811646e-02,2.182449e-02,1.498828e-02,1.442741e-02,2.056325e-02,2.350256e-02,4.434756e-02,9.740444e-01,4.599567e-02,7.327345e-02,1.749811e+00,7.769020e-01,7.514681e-02,6.147144e-02,3.408556e-02,1.021748e-01,5.919538e-01,1.691483e-01,4.774818e-01,3.158906e-01,2.241519e+01,9.699106e+00,1.218716e-01,5.461300e-01,1.551633e-01,6.601308e+00,1.603838e+00,1.776434e-02,1.817005e+00,1.656500e+00,2.383889e-01,1.608228e+01,2.095249e+00,9.190984e-02,8.526462e-02,8.346494e-02,1.897165e+01,4.018624e-01,4.065377e-02,1.219073e-03,7.710093e-04,6.058669e-03,1.608065e-02,1.128592e+00,2.717994e-01,3.770186e+00,6.371737e+00,3.166550e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,-0.000000e+00,2.777778e-01,0.000000e+00,1.250000e-01,1.250000e-01,4.949956e-02,1.420164e+00,1.702820e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.325942e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00

In [15]:
df.label.value_counts()

label
0    5283175
1    1445673
Name: count, dtype: int64